# Tests for DataMux

In [1]:
import asyncio
import logging
from dotenv import load_dotenv
from rich.logging import RichHandler

logging.basicConfig(level=logging.INFO, format="%(message)s", datefmt="[%X]", handlers=[RichHandler()])
load_dotenv()

True

In [2]:
from datamux.remote.api import DataMuxRemoteAPI

server_host = "localhost"
server_port = 3300

api = DataMuxRemoteAPI(rpc_name="websocket", codec_name="avro")
await api.connect(server_host, server_port)

INFO:rpc.client.websocket:Connected to WebSocket Server: ws://localhost:3300/ws


In [3]:
collections = await api.list_collections()
for collection in collections:
    print(collection)

name='pg_weather' description='Weather Dataset' keywords=['Weather', 'Climate'] authors=[Author(name='Gabriel Dzodom', affiliation='Texas A&M University', email='gabriel.dzodom@tamu.edu'), Author(name='Frank Shipman', affiliation='Texas A&M University', email='shipman@cse.tamu.edu')] streams={'temperature': Stream(name='temperature', description='temperature', unit='F', frequency=1.0, fields={'maxtemperature': Field(name='max_temperature', description='max temperature', dtype=<class 'numpy.float32'>), 'meantemperature': Field(name='mean_temperature', description='mean temperature', dtype=<class 'numpy.float32'>), 'mintemperature': Field(name='min_temperature', description='min temperature', dtype=<class 'numpy.float32'>)}, index={'t': Field(name='date', description='date', dtype=<class 'numpy.int32'>)}, node=None, attrs={}), 'dewpoint': Stream(name='dewpoint', description='dewpoint', unit='', frequency=1.0, fields={'maxdewpoint': Field(name='max_dewpoint', description='max dewpoint', d

In [4]:
collection_streams = await api.list_collection_streams("adhd_sin")
for stream in collection_streams:
    print(stream)

name='Gaze' description='gaze point' unit='' frequency=60.0 fields={'x': Field(name='gaze x', description='Normalized X-Gaze Point', dtype=<class 'numpy.float32'>), 'y': Field(name='gaze y', description='Normalized Y-Gaze Point', dtype=<class 'numpy.float32'>)} index={'t': Field(name='timestamp', description='Recording Timestamp', dtype=<class 'numpy.float32'>)} node=None attrs={'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}
name='Pupil' description='pupil diameter' unit='mm' frequency=60.0 fields={'d': Field(name='pupil d', description='Pupil Diameter', dtype=<class 'numpy.float32'>)} index={'t': Field(name='timestamp', description='Recording Timestamp', dtype=<class 'numpy.float32'>)} node=None attrs={'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'pupil'}
name='Gaze' description='gaze point' unit='' frequency=60

In [5]:
collection_name = "adhd_sin"
stream_name = "Gaze"
attrs = dict(
    {
        "diagnosis": "Non-ADHD",
        "gender": "female",
        "noise": "0",
        "participant": "12",
        "question": "10",
        "collection": "adhd_sin",
        "id": "gaze",
    }
)

In [6]:
sink = asyncio.Queue()
ack = await api.replay_collection_stream(collection_name, stream_name, attrs, sink)
print(ack)
for _ in range(25):
    msg = await sink.get()
    print(msg)

status=True randseq='ofZqo'
{'index': {'t': 16.670000076293945}, 'value': {'x': 976.0, 'y': 752.0}}
{'index': {'t': 33.33000183105469}, 'value': {'x': 975.0, 'y': 750.0}}
{'index': {'t': 50.0}, 'value': {'x': 973.0, 'y': 748.0}}
{'index': {'t': 66.66000366210938}, 'value': {'x': 972.0, 'y': 745.0}}
{'index': {'t': 83.33000183105469}, 'value': {'x': 966.0, 'y': 747.0}}
{'index': {'t': 99.95999908447266}, 'value': {'x': 970.0, 'y': 753.0}}
{'index': {'t': 116.62999725341797}, 'value': {'x': 969.0, 'y': 747.0}}
{'index': {'t': 133.2899932861328}, 'value': {'x': 976.0, 'y': 744.0}}
{'index': {'t': 149.94000244140625}, 'value': {'x': 969.0, 'y': 753.0}}
{'index': {'t': 166.6199951171875}, 'value': {'x': 968.0, 'y': 745.0}}
{'index': {'t': 183.2899932861328}, 'value': {'x': 971.0, 'y': 748.0}}
{'index': {'t': 199.92999267578125}, 'value': {'x': 971.0, 'y': 739.0}}
{'index': {'t': 216.61000061035156}, 'value': {'x': 976.0, 'y': 746.0}}
{'index': {'t': 233.27999877929688}, 'value': {'x': 981.0

In [7]:
ack = await api.publish_collection_stream(collection_name, stream_name, attrs)
print(ack)
await asyncio.sleep(5)

status=True randseq='QGsao'


In [8]:
live_streams = await api.list_live_streams()
for stream in live_streams:
    print(stream)

name='Gaze' description='gaze point' unit='' frequency=60.0 fields={'x': Field(name='gaze x', description='Normalized X-Gaze Point', dtype=<class 'numpy.float32'>), 'y': Field(name='gaze y', description='Normalized Y-Gaze Point', dtype=<class 'numpy.float32'>)} index={'t': Field(name='timestamp', description='Recording Timestamp', dtype=<class 'numpy.float32'>)} node=None attrs={'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze', 'dfds_mode': 'restream'}


In [9]:
assert len(live_streams) > 0
live_stream = live_streams[0]

sink = asyncio.Queue()
ack = await api.read_live_stream(live_stream.name, live_stream.attrs, sink)
print(ack)
for _ in range(25):
    msg = await sink.get()
    print(msg)

status=True randseq='czwuY'
{'index': {'t': 20369.1328125}, 'value': {'x': 973.0, 'y': 748.0}}
{'index': {'t': 16.670000076293945}, 'value': {'x': 976.0, 'y': 752.0}}
{'index': {'t': 33.33000183105469}, 'value': {'x': 975.0, 'y': 750.0}}
{'index': {'t': 50.0}, 'value': {'x': 973.0, 'y': 748.0}}
{'index': {'t': 66.66000366210938}, 'value': {'x': 972.0, 'y': 745.0}}
{'index': {'t': 83.33000183105469}, 'value': {'x': 966.0, 'y': 747.0}}
{'index': {'t': 99.95999908447266}, 'value': {'x': 970.0, 'y': 753.0}}
{'index': {'t': 116.62999725341797}, 'value': {'x': 969.0, 'y': 747.0}}
{'index': {'t': 133.2899932861328}, 'value': {'x': 976.0, 'y': 744.0}}
{'index': {'t': 149.94000244140625}, 'value': {'x': 969.0, 'y': 753.0}}
{'index': {'t': 166.6199951171875}, 'value': {'x': 968.0, 'y': 745.0}}
{'index': {'t': 183.2899932861328}, 'value': {'x': 971.0, 'y': 748.0}}
{'index': {'t': 199.92999267578125}, 'value': {'x': 971.0, 'y': 739.0}}
{'index': {'t': 216.61000061035156}, 'value': {'x': 976.0, 'y'